In [1]:
#极坐标弧长曲线求解部分
from scipy import integrate
import sympy as sp
import numpy as np


# 定义符号变量theta
theta = sp.symbols('theta')

#化简积分表达式
c = 55 / (2 * sp.pi)

# 弧长的不定积分,单位为cm
int_expr = 0.5 * c * (sp.sqrt(1 + theta**2) * theta + sp.asinh(theta))

int_expr


13.75*(theta*sqrt(theta**2 + 1) + asinh(theta))/pi

In [2]:

def arc(t0,arc_length) -> float:
    '''
    输入当前的角度t0和弧长arc_length
    返回前进到的角度t1
    积分是从t1到t0，因为t1的角度更小,前进过程中角度是在减小的
    '''
    a =  0.5 * c * (sp.sqrt(1 + t0**2) * t0 + sp.asinh(t0))
    t1 = sp.symbols('t1') # 定义符号变量t1,设定为未知数
    b =  0.5 * c * (sp.sqrt(1 + t1**2) * t1 + sp.asinh(t1))
    
    equation = a -b - arc_length
    #sp.pprint(equation)
    result = sp.nsolve(equation,t1, t0)
    
    return float(result)

'''
当前角度为32π，沿着弧线前进2cm
print(arc(32*np.pi,1))
'''

class Queue:
    '''
    实现一个队列，先入先出(FIFO)
    '''
    def __init__(self):
        self.queue = []
    
    def is_empty(self): 
        return len(self.queue) == 0
    
    def push(self, element):
        self.queue.append(element)
    
    def pop(self):
        if self.is_empty():
            return None
        else:
            return self.queue.pop(0)
    
    def get_size(self):
        return len(self.queue)

        
def search_vertex(head_theta) -> list:
    c = 55 / (2 * np.pi)  # 常数 c，用来计算螺线坐标
    theta_step = 0.0005     # 角度步长
    vertex = Queue()      # 队列，用于存储待处理的节点
    vertex_list = []      # 存储所有找到的顶点角度
    
    # 初始角度作为第一个顶点
    vertex.push(head_theta)
    vertex_list.append(head_theta)
    while not vertex.is_empty():
        if len(vertex_list) > 223: # 限制最多只能找到 224 个顶点
            print("找完了",len(vertex_list))
            break
        
        # 从队列中取出当前的角度（作为线段的固定端点）
        current_theta = vertex.pop()
        #print(f"Processing theta: {current_theta}")
        
        # 当前点的坐标
        x0 = c * current_theta * np.cos(current_theta)
        y0 = c * current_theta * np.sin(current_theta)
        
        # 从当前 theta 开始，移动下一个点
        theta = current_theta
        while theta <= 32 * np.pi:
            # 尝试计算下一个点的角度
            next_theta = theta + theta_step
            x1 = c * next_theta * np.cos(next_theta)
            y1 = c * next_theta * np.sin(next_theta)
            
            # 计算两点之间的距离
            distance = np.sqrt((x1 - x0) ** 2 + (y1 - y0) ** 2)
            if distance > 890:
                print(f"Warning!!!!! Distance too large: {distance}")
            
            # 处理距离为 286 的情况，且这是寻找“头部”的条件
            if distance >= 286 and len(vertex_list) == 1:
                #print("ok")
                #print(f"Found head: {next_theta} with distance: {distance}")
                vertex_list.append(next_theta)
                vertex.push(next_theta)  # 将新找到的顶点作为新的固定端点继续搜索
                break  # 找到符合条件的点后跳出当前循环，开始处理下一个顶点
            
            # 处理距离为 165 的情况
            if distance >= 165 and len(vertex_list) > 1 :
                #print(f"Found vertex at theta: {next_theta} with distance: {distance}")
                vertex_list.append(next_theta)
                vertex.push(next_theta)  # 将新找到的顶点作为新的固定端点继续搜索
                break  # 找到符合条件的点后跳出当前循环，开始处理下一个顶点
            
            # 更新 theta，继续向下搜索
            theta += theta_step
    
    return vertex_list
'''
# 测试函数
found_vertices = search_vertex(0.0)
print("Found vertices:", found_vertices)
'''
def distance(a,b) -> float:
    '''
    输入两个角度a和b
    用近似替代方法计算两点之间的距离
    '''
    c = 55 / (2 * np.pi)
    
    x0 = c * a * np.cos(a)
    y0 = c * a * np.sin(a)
    x1 = c * b * np.cos(b)
    y1 = c * b * np.sin(b)
    #print("x0,y0,x1,y1",x0,y0,x1,y1)
    return np.sqrt((x1 - x0) ** 2 + (y1 - y0) ** 2)
#distance(32.67299999997398, 33.25299999997675)

def get_speed(t0,time):
    '''
    输入初始角度和运动时间time
    确定时间微元dt为1e-7
    返回进入螺线的所有节点的坐标和速度
    '''
    dt = 1e-7
    
    current_arc = time*100 #当前头部的弧长
    updated_arc = (time+dt)*100 #dt时间内头部加上走过的弧长
    
    head_theta = arc(t0,current_arc) #头部当前的角度
    updated_head_theta = arc(t0,updated_arc) #dt后头部的角度
    
    current_theta_list = search_vertex(head_theta)
    #print(current_theta_list)
    after_theta_list = search_vertex(updated_head_theta)  

    
    speed_list = []
    for i in range(len(current_theta_list)):
        a = current_theta_list[i]
        b = after_theta_list[i]
        ds = distance(a,b)
        speed = ds/dt
        speed_list.append(speed)
        
    return current_theta_list,speed_list
    

In [3]:
test = []
def crash_judge(thetaA,thetaE,thetaP,thetaQ) -> bool:
    '''
    theta为某一节龙身的把手的角度，判断是否与龙头的把手相撞
    '''  
    c = 55 / (2 * np.pi)
      
    xA = c * thetaA * np.cos(thetaA)
    yA = c * thetaA * np.sin(thetaA)
    xE = c * thetaE * np.cos(thetaE)
    yE = c * thetaE * np.sin(thetaE)
    
    k_AE = (yA - yE) / (xA - xE) 
    k_AE_vertical = -1 / k_AE
    
    d = yA - k_AE * xA
    b1 = d + 15*np.sqrt(1 + k_AE**2)
    b2 = d - 15*np.sqrt(1 + k_AE**2)
    
    c1 = yA - k_AE_vertical * xA +27.5*np.sqrt(1 + k_AE_vertical**2)
    c2 = yA - k_AE_vertical * xA -27.5*np.sqrt(1 + k_AE_vertical**2)
    
    xB1 = (c1 - b1) / (k_AE - k_AE_vertical)
    xB2 = (c1 - b2) / (k_AE - k_AE_vertical)
    xB3 = (c2 - b1) / (k_AE - k_AE_vertical)
    xB4 = (c2 - b2) / (k_AE - k_AE_vertical)
    
    yB1 = k_AE * xB1 + b1
    yB2 = k_AE * xB2 + b2
    yB3 = k_AE * xB3 + b1
    yB4 = k_AE * xB4 + b2
    
    xP = c * thetaP * np.cos(thetaP)
    yP = c * thetaP * np.sin(thetaP)
    xQ = c * thetaQ * np.cos(thetaQ)
    yQ = c * thetaQ * np.sin(thetaQ)
    
    k_PQ = (yP - yQ) / (xP - xQ)
    
    m = yP - k_PQ * xP
    
    #取PQ中点
    xS = (xP + xQ) / 2
    yS = (yP + yQ) / 2
    
    
    #对得到的四个点进行判断
    j1 = np.abs(k_PQ * xB1 - yB1 + m) / np.sqrt(k_PQ**2 + 1)
    j2 = np.abs(k_PQ * xB2 - yB2 + m) / np.sqrt(k_PQ**2 + 1)
    j3 = np.abs(k_PQ * xB3 - yB3 + m) / np.sqrt(k_PQ**2 + 1)
    j4 = np.abs(k_PQ * xB4 - yB4 + m) / np.sqrt(k_PQ**2 + 1)
    test_list = [j1,j2,j3,j4] #测试用
    d1 = np.sqrt((xS - xB1)**2 + (yS - yB1)**2)
    d2 = np.sqrt((xS - xB2)**2 + (yS - yB2)**2)
    d3 = np.sqrt((xS - xB3)**2 + (yS - yB3)**2)
    d4 = np.sqrt((xS - xB4)**2 + (yS - yB4)**2) 

    if j1 <= 15 and d1 <= 111 :
        return True

    if j2 <= 15 and d2 <= 111 :
        return True
    
    if j3 <= 15 and d3 <= 111 :
        return True
    
    if j4 <= 15 and d4 <= 111 :
        return True
    
    test.append(min(test_list))
    return False


In [4]:
import numpy as np

def main_q2(time):
    theta_list , speed_list = get_speed(32*np.pi,time)
    try:
        head_A = theta_list[0]
        head_B = theta_list[1]
        for i in range(2,len(theta_list)):
            body_P = theta_list[i]
            body_Q = theta_list[i+1]
            
            print("searching time:",time)
            if crash_judge(head_A,head_B,body_P,body_Q): 
                print(f"Crash!!!,Time At:{time}")
                print(f"current theta head_A:{head_A},head_B:{head_B},body_P:{body_P},body_Q:{body_Q}")
                return True
    except IndexError:
        return False
        
        
for i in np.arange(1,500,1):
    main_q2(i)
    if main_q2(i):
        print("OK")
        break    

searching time: 7
searching time: 7
searching time: 8
searching time: 8
searching time: 8
searching time: 8
searching time: 9
searching time: 9
searching time: 9
searching time: 9
searching time: 10
searching time: 10
searching time: 10
searching time: 10
searching time: 10
searching time: 10
searching time: 11
searching time: 11
searching time: 11
searching time: 11
searching time: 11
searching time: 11
searching time: 12
searching time: 12
searching time: 12
searching time: 12
searching time: 12
searching time: 12
searching time: 12
searching time: 12
searching time: 13
searching time: 13
searching time: 13
searching time: 13
searching time: 13
searching time: 13
searching time: 13
searching time: 13
searching time: 13
searching time: 13
searching time: 14
searching time: 14
searching time: 14
searching time: 14
searching time: 14
searching time: 14
searching time: 14
searching time: 14
searching time: 14
searching time: 14
searching time: 15
searching time: 15
searching time: 15
sea

In [5]:
def main_q2(time):
    theta_list , speed_list = get_speed(32*np.pi,time)
    print(len(theta_list),theta_list)
    print(len(speed_list),speed_list)
    x_list = []
    y_list = []
    target_speed = []
    for i in range(len(theta_list)):
            temp = theta_list[i]
            x = 55 / (2 * np.pi) * temp * np.cos(temp)
            y = 55 / (2 * np.pi) * temp * np.sin(temp)
            print(f"theta: {temp}, x: {x}, y: {y}")
            x_list.append(x)
            y_list.append(y)
            target_speed.append(speed_list[i])
            
    print("x:",x_list)
    print("y:",y_list)
    print("speed:",target_speed)

    import pandas as pd
    df = pd.DataFrame({"x":[o/100 for o in x_list],"y":[p/100 for p in y_list],"speed":[q/100 for q in target_speed]})
    df.to_csv(f"q2_crash_{time}_all.csv",index=True)
    
main_q2(413)


找完了 224
找完了 224
224 [25.91579464644424, 27.239294646441156, 27.93629464643953, 28.615794646437948, 29.278794646436403, 29.926794646434892, 30.560294646433416, 31.18079464643197, 31.788794646430553, 32.3847946464319, 32.96979464643469, 33.544294646437436, 34.10879464644013, 34.664294646442784, 35.21079464644539, 35.74879464644796, 36.27829464645049, 36.80029464645298, 37.31479464645544, 37.82179464645786, 38.32229464646025, 38.81629464646261, 39.30379464646494, 39.785294646467236, 40.260794646469506, 40.73079464647175, 41.19529464647397, 41.65429464647616, 42.10829464647833, 42.55729464648047, 43.001794646482594, 43.441794646484695, 43.877294646486774, 44.30829464648883, 44.73529464649087, 45.15779464649289, 45.57629464649489, 45.99129464649687, 46.40229464649883, 46.80979464650078, 47.213794646502706, 47.61429464650462, 48.011294646506514, 48.40479464650839, 48.79529464651026, 49.18279464651211, 49.56729464651394, 49.948794646515765, 50.32729464651757, 50.702794646519365, 51.0752946465